# KNN 1

In [1]:
import pandas as pd

Rcols = ['user_id', 'movie_id', 'rating']
Ratings = pd.read_csv("C:\\Users\\Admin\\OneDrive\\Documents\\Python\\Assignment\\knn\\KNearest Neighbours\\u.data", sep='\t', names=Rcols, usecols=range(3))
Ratings.head()

,user_id,movie_id,rating
0,0,50,5
1,0,172,5
2,0,133,1
3,196,242,3
4,186,302,3


Now, we'll group everything by movie ID, and compute the total number of ratings (each movie's popularity) and the average rating for every movie

In [2]:
import numpy as np

movieProperties = Ratings.groupby('movie_id').agg({'rating': [np.size, np.mean]})
movieProperties.head()

rating          
           size      mean
movie_id                 
1           452  3.878319
2           131  3.206107
3            90  3.033333
4           209  3.550239
5            86  3.302326

The raw number of ratings isn't very useful for computing distances between movies, so we'll create a new DataFrame that contains the normalized number of ratings. So, a value of 0 means nobody rated it, and a value of 1 will mean it's the most popular movie there is.

In [3]:
movieNumRatings = pd.DataFrame(movieProperties['rating']['size'])
movieNormalizedNumRatings = movieNumRatings.apply(lambda x: (x - np.min(x)) / (np.max(x) - np.min(x)))
movieNormalizedNumRatings.head()

,size
movie_id,
1,0.773585
2,0.222985
3,0.152659
4,0.356775
5,0.145798


Now, let's get the genre information from the u.item file. The way this works is there are 19 fields, each corresponding to a specific genre - a value of '0' means it is not in that genre, and '1' means it is in that genre. A movie may have more than one genre associated with it.

While we're at it, we'll put together everything into one big Python dictionary called movieDict. Each entry will contain the movie name, list of genre values, the normalized popularity score, and the average rating for each movie:

In [4]:
movieDict = {}
with open(r"C:\\Users\\Admin\\OneDrive\\Documents\\Python\\Assignment\\knn\\KNearest Neighbours\\u.item") as f:
    temp = ''
    for line in f:
        
        fields = line.rstrip('\n').split('|')
        movieID = int(fields[0])
        name = fields[1]
        genres = fields[5:25]
        genres = map(int, genres)
        movieDict[movieID] = (name, np.array(list(genres)), movieNormalizedNumRatings.loc[movieID].get('size'), movieProperties.loc[movieID].rating.get('mean'))


In [5]:
print(movieDict[786])

('Perez Family, The (1995)', array([0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0]), 0.022298456260720412, 2.642857142857143)


Now let's define a function that computes the "distance" between two movies based on how similar their genres are, and how similar their popularity is. Just to make sure it works, we'll compute the distance between movie ID's 2 and 4:

In [7]:
from scipy import spatial

def ComputeDistance(a, b):
    genresA = a[1]
    genresB = b[1]
    genreDistance = spatial.distance.cosine(genresA, genresB)
    popularityA = a[2]
    popularityB = b[2]
    popularityDistance = abs(popularityA - popularityB)
    return genreDistance + popularityDistance
    
ComputeDistance(movieDict[143], movieDict[786])


1.3567753001715266

In [8]:
print(movieDict[143])
print(movieDict[786])


('Sound of Music, The (1965)', array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0]), 0.379073756432247, 3.765765765765766)
('Perez Family, The (1995)', array([0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0]), 0.022298456260720412, 2.642857142857143)


In [9]:
import operator

def getNeighbors(movieID, K):
    distances = []
    for movie in movieDict:
        if (movie != movieID):
            dist = ComputeDistance(movieDict[movieID], movieDict[movie])
            distances.append((movie, dist))
    distances.sort(key=operator.itemgetter(1))
    neighbors = []
    for x in range(K):
        neighbors.append(distances[x][0])
    return neighbors

K = 7
avgRating = 0
neighbors = getNeighbors(786, K)
for neighbor in neighbors:
    avgRating += movieDict[neighbor][3]
    print (movieDict[neighbor][0] + " " + str(movieDict[neighbor][3]))
    
avgRating /= K

When the Cats Away (Chacun cherche son chat) (1996) 3.5625
Vampire in Brooklyn (1995) 1.8333333333333333
That Old Feeling (1997) 3.3636363636363638
Love in the Afternoon (1957) 3.9
Pyromaniac's Love Story, A (1995) 2.4285714285714284
Two Much (1996) 2.4285714285714284
Nina Takes a Lover (1994) 2.8333333333333335


In [10]:
avgRating

2.907135126777984

In [11]:
movieDict[786]

('Perez Family, The (1995)',
 array([0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0]),
 0.022298456260720412,
 2.642857142857143)